In [2]:
# # Import Supporting Libraries
# import pandas as pd

# # Import Dash Visualization Libraries
# import dash_core_components as dcc
# import dash_html_components as html
# import dash_table_experiments as dt
# import dash.dependencies
# from dash.dependencies import Input, Output, State
# import plotly


# # Load datasets
# US_STATES_URL = 'https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv'
# US_AG_URL = 'https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv'
# df_st = pd.read_csv(US_STATES_URL)
# df_ag = pd.read_csv(US_AG_URL)
# print(df_st.head())
# print(df_ag.head())


# # Create our app layout
# app = dash.Dash(__name__)
# server = app.server
# app.layout = html.Div([
#     html.H2('My Dash App'),
#     dt.DataTable(
#         id='my-datatable',
#         rows=df_ag.to_dict('records'),
#         editable=False,
#         row_selectable=True,
#         filterable=True,
#         sortable=True,
#         selected_row_indices=[]
#     ),
#     html.Div(id='selected-indexes'),
#     dcc.Graph(
#         id='datatable-subplots'
#     )
# ], style={'width': '60%'})

# dcc._css_dist[0]['relative_package_path'].append('local.css')

# @app.callback(Output('my-datatable', 'selected_row_indices'),
#               [Input('datatable-subplots', 'clickData')],
#               [State('my-datatable', 'selected_row_indices')])
# def updated_selected_row_indices(clickData, selected_row_indices):
#     if clickData:
#         for point in clickData['points']:
#             if point['pointNumber'] in selected_row_indices:
#                 selected_row_indices.remove(point['pointNumber'])
#             else:
#                 selected_row_indices.append(point['pointNumber'])
#     return selected_row_indices


# @app.callback(Output('datatable-subplots', 'figure'),
#               [Input('my-datatable', 'rows'),
#                Input('my-datatable', 'selected_row_indices')])
# def update_figure(rows, selected_row_indices):
#     dff = pd.DataFrame(rows)
#     fig = plotly.tools.make_subplots(
#         rows=3, cols=1,
#         subplot_titles=('Beef', 'Pork', 'Poultry'),
#         shared_xaxes=True)
#     marker = {'color': ['#0074D9']*len(dff)}
#     for i in (selected_row_indices or []):
#         marker['color'][i] = '#FF851B'
#     fig.append_trace({
#         'x': dff['state'],
#         'y': dff['beef'],
#         'type': 'bar',
#         'marker': marker
#     }, 1, 1)
#     fig.append_trace({
#         'x': dff['state'],
#         'y': dff['pork'],
#         'type': 'bar',
#         'marker': marker
#     }, 2, 1)
#     fig.append_trace({
#         'x': dff['state'],
#         'y': dff['poultry'],
#         'type': 'bar',
#         'marker': marker
#     }, 3, 1)
#     fig['layout']['showlegend'] = False
#     fig['layout']['height'] = 1000
#     fig['layout']['width'] = 1200
#     fig['layout']['margin'] = {
#         'l': 40,
#         'r': 10,
#         't': 60,
#         'b': 200
#     }
#     return fig


# app.css.append_css({
#     'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
# })


# if __name__ == '__main__':
#     app.run_server(debug=False)

In [2]:
# Imports
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objs as go
import datetime

In [42]:
#Load and preprocess data
# df = pd.read_csv(
#     'https://gist.githubusercontent.com/chriddyp/'
#     'cb5392c35661370d95f300086accea51/raw/'
#     '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
#     'indicators.csv')

# available_indicators = df['Indicator Name'].unique()

df_pp = pd.read_csv('/home/deaddemocracy/Documents/Major_Project/Extract_Feb18_28_1PM/sensor1/sensor1_pp.csv', decimal = ',', index_col = 0)
available_indicators = list(df_pp.columns.values)


In [45]:
df_pp.index = pd.to_datetime(df_pp.index)

In [46]:
available_indicators

['ax(g)',
 'ay(g)',
 'az(g)',
 'wx(deg/s)',
 'wy(deg/s)',
 'wz(deg/s)',
 'AngleX(deg)',
 'AngleY(deg)',
 'AngleZ(deg)',
 'T(°)',
 'hx',
 'hy',
 'hz']

In [44]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

In [41]:

# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([

        html.Div([
#             dcc.Dropdown(
#                 id='xaxis-column',
#                 options=[{'label': i, 'value': i} for i in available_indicators],
#                 value='Fertility rate, total (births per woman)'
#             ),
#             dcc.RadioItems(
#                 id='xaxis-type',
#                 options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
#                 value='Linear',
#                 labelStyle={'display': 'inline-block'}
#             )
        ],
        style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),

    dcc.Graph(id='indicator-graphic'),

    dcc.Slider(
        id='year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    )
])

# Callbacks
@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('xaxis-column', 'value'),
     dash.dependencies.Input('yaxis-column', 'value'),
     dash.dependencies.Input('xaxis-type', 'value'),
     dash.dependencies.Input('yaxis-type', 'value'),
     dash.dependencies.Input('year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]
    return {
        'data': [go.Scatter(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 15,
                'opacity': 1,
                'color': 'blue',
                'line': {'width': 2}
            }
        )],
        'layout': go.Layout(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest',
        )
    }

viewer.show(app)